In [1]:

from app.log import logging_config
from os.path import dirname, join
from dotenv import load_dotenv
import logging
from app.util.message import starting_message
from app import Polygon
from app import Sentiment_predict
import pandas as pd


c:\Users\andre\Anaconda3\envs\DS4A_PF\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Polygon

In [2]:
load_dotenv()
from os import environ
print(environ.get("POLYGON_KEY"))
logging_config.init_logging()
__LOG = logging.getLogger(__name__)
__LOG.info('...... Initialization Completed  ......')
__LOG.info(starting_message())

PWRjSfmBc9xYnPZMQhA9ukDOuOIId90D


In [3]:
polygon = Polygon(environ.get("POLYGON_KEY"))

In [4]:
status, polygon_data = polygon.get_news('')
if status:
    polygon_news_df = pd.DataFrame.from_dict(polygon_data['results'])
polygon_news_df

,id,publisher,title,author,published_utc,article_url,tickers,amp_url,image_url,description,keywords
0,jhv4UeEySuDXFduebwCGxGedWm18KS4w3xt33U93Dm8,"{'name': 'Invezz', 'homepage_url': 'https://in...",Carnival stock could be worth “$0” in the wors...,Wajeeh Khan,2022-06-29T18:58:27Z,https://invezz.com/news/2022/06/29/carnival-st...,[CCL],https://invezz.com/news/2022/06/29/carnival-st...,https://invezz.com/wp-content/uploads/2022/06/...,Carnival Corp (NYSE: CCL) is down 15% on Wedne...,"[Industries, North America, Stocks & Shares, T..."
1,51LMiiZ4R9p8njD2LeXOIS5N89ANEDJJyBag4BNx71A,"{'name': 'The Motley Fool', 'homepage_url': 'h...",Why Rivian Shares Are Down Again Today,newsfeedback@fool.com (Howard Smith),2022-06-29T18:45:07Z,https://www.fool.com/investing/2022/06/29/why-...,[RIVN],NaN,https://g.foolcdn.com/editorial/images/687630/...,Rivian opened its first three fast charging si...,[investing]
2,QogvcVOH_Ma-q1t9R0Aa8CAT7Uk6VDC_wxnYaqNcvIk,"{'name': 'MarketWatch', 'homepage_url': 'https...",Novavax is part of the FDA's COVID-19 booster ...,MarketWatch,2022-06-29T18:45:00Z,https://www.marketwatch.com/story/novavax-is-p...,[NVAX],https://www.marketwatch.com/amp/story/novavax-...,https://images.mktw.net/im-574104/social,The experimental shot played a front-and-cente...,NaN
3,jXltVMLRQ0pY6Cjc7XQwkCgxVkXLJzFWpYsJdhE8utc,"{'name': 'MarketWatch', 'homepage_url': 'https...",Challenges facing ousted Bed Bath & Beyond CEO...,MarketWatch,2022-06-29T18:43:00Z,https://www.marketwatch.com/story/challenges-f...,"[BBBY, TGT, AMZN, WMT, SPG]",https://www.marketwatch.com/amp/story/challeng...,https://images.mktw.net/im-524957/social,"Mark Tritton, who has been ousted as Bed Bath ...",NaN
4,TpQ9cnB3hZI4S1Sy1hgKX_QAxwze2Mme_-Rqah9T4Bw,"{'name': 'PennyStocks', 'homepage_url': 'https...",Best Penny Stocks to Buy Right Now? 3 to Watch...,J. Phillip,2022-06-29T18:33:42Z,https://pennystocks.com/featured/2022/06/29/be...,"[BHAT, HTGM, TOUR]",https://pennystocks.com/featured/2022/06/29/be...,https://pennystocks.com/wp-content/uploads/202...,Which penny stocks are you watching right now?...,"[Featured, Penny Stock News, Trading Penny Sto..."
...,...,...,...,...,...,...,...,...,...,...,...
995,K4A7-a5NNRNFT0HnMzpiV68dVpAUX5YmG71Ap5B00vU,"{'name': 'Zacks Investment Research', 'homepag...",Bear Of The Day: Autodesk (ADSK),Brian Bolan,2022-06-28T12:16:00Z,https://www.zacks.com/commentary/1944646/bear-...,[ADSK],https://www.zacks.com/amp/commentary/1944646/b...,https://staticx-tuner.zacks.com/images/default...,Despite beating the Zacks Consensus Estimate i...,NaN
996,mJVdMewKY20b0FJzPXDHFQ6-pLZNVJ36z5vZjpdhXE0,"{'name': 'Zacks Investment Research', 'homepag...","Cognyte Software Ltd. (CGNT) Reports Q1 Loss, ...",Zacks Equity Research,2022-06-28T12:15:02Z,https://www.zacks.com/stock/news/1944857/cogny...,"[CGNT, TENB]",https://www.zacks.com/amp/stock/news/1944857/c...,https://staticx-tuner.zacks.com/images/default...,Cognyte Software Ltd. (CGNT) delivered earning...,NaN
997,DBfrpkBx33T7oiptfiJHrLZXKaXlytJ3cuXhaWcd6ps,"{'name': 'The Motley Fool', 'homepage_url': 'h...",3 High-Yield REITs That Have Made Me a Ton of ...,newsfeedback@fool.com (Liz Brumer-Smith),2022-06-28T12:15:00Z,https://www.fool.com/investing/2022/06/28/3-hi...,"[PLYM, IIPR, FRT]",NaN,https://g.foolcdn.com/editorial/images/686902/...,The power of dividend growth is severely under...,[investing]
998,t4RR4xsv1VZU_2xycp7E9ihRyFNAZSxa1hppRVUOUow,"{'name': 'MarketWatch', 'homepage_url': 'https...","Goldman, Wells Fargo lead dividend buffet afte...",MarketWatch,2022-06-28T12:14:00Z,https://www.marketwatch.com/story/goldman-well...,"[WFC, GS, MS, BAC, C, JPM, DJIA, XLF]",https://www.marketwatch.com/amp/story/goldman-...,https://images.mktw.net/im-572781/social,Megabanks reveal shareholder payout plans afte...,NaN


In [5]:
sp = Sentiment_predict()

In [7]:
polygon_news_df['title_sent_finBERT'] = sp.sentiment_finBERT(polygon_news_df, 'title')
polygon_news_df['description_finBERT'] = sp.sentiment_finBERT(polygon_news_df, 'description')

polygon_news_df['title_sent_financialBERT'] = sp.sentiment_financialBERT(polygon_news_df, 'title')
polygon_news_df['description_financialBERT'] = sp.sentiment_financialBERT(polygon_news_df, 'description')

Token indices sequence length is longer than the specified maximum sequence length for this model (544 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (671 > 512). Running this sequence through the model will result in indexing errors


In [8]:
polygon_news_df

,id,publisher,title,author,published_utc,article_url,tickers,amp_url,image_url,description,keywords,title_sent_finBERT,description_finBERT,title_sent_financialBERT,description_financialBERT
0,jhv4UeEySuDXFduebwCGxGedWm18KS4w3xt33U93Dm8,"{'name': 'Invezz', 'homepage_url': 'https://in...",Carnival stock could be worth “$0” in the wors...,Wajeeh Khan,2022-06-29T18:58:27Z,https://invezz.com/news/2022/06/29/carnival-st...,[CCL],https://invezz.com/news/2022/06/29/carnival-st...,https://invezz.com/wp-content/uploads/2022/06/...,Carnival Corp (NYSE: CCL) is down 15% on Wedne...,"[Industries, North America, Stocks & Shares, T...",neutral,negative,neutral,negative
1,51LMiiZ4R9p8njD2LeXOIS5N89ANEDJJyBag4BNx71A,"{'name': 'The Motley Fool', 'homepage_url': 'h...",Why Rivian Shares Are Down Again Today,newsfeedback@fool.com (Howard Smith),2022-06-29T18:45:07Z,https://www.fool.com/investing/2022/06/29/why-...,[RIVN],NaN,https://g.foolcdn.com/editorial/images/687630/...,Rivian opened its first three fast charging si...,[investing],negative,neutral,neutral,neutral
2,QogvcVOH_Ma-q1t9R0Aa8CAT7Uk6VDC_wxnYaqNcvIk,"{'name': 'MarketWatch', 'homepage_url': 'https...",Novavax is part of the FDA's COVID-19 booster ...,MarketWatch,2022-06-29T18:45:00Z,https://www.marketwatch.com/story/novavax-is-p...,[NVAX],https://www.marketwatch.com/amp/story/novavax-...,https://images.mktw.net/im-574104/social,The experimental shot played a front-and-cente...,NaN,neutral,neutral,neutral,neutral
3,jXltVMLRQ0pY6Cjc7XQwkCgxVkXLJzFWpYsJdhE8utc,"{'name': 'MarketWatch', 'homepage_url': 'https...",Challenges facing ousted Bed Bath & Beyond CEO...,MarketWatch,2022-06-29T18:43:00Z,https://www.marketwatch.com/story/challenges-f...,"[BBBY, TGT, AMZN, WMT, SPG]",https://www.marketwatch.com/amp/story/challeng...,https://images.mktw.net/im-524957/social,"Mark Tritton, who has been ousted as Bed Bath ...",NaN,negative,negative,negative,negative
4,TpQ9cnB3hZI4S1Sy1hgKX_QAxwze2Mme_-Rqah9T4Bw,"{'name': 'PennyStocks', 'homepage_url': 'https...",Best Penny Stocks to Buy Right Now? 3 to Watch...,J. Phillip,2022-06-29T18:33:42Z,https://pennystocks.com/featured/2022/06/29/be...,"[BHAT, HTGM, TOUR]",https://pennystocks.com/featured/2022/06/29/be...,https://pennystocks.com/wp-content/uploads/202...,Which penny stocks are you watching right now?...,"[Featured, Penny Stock News, Trading Penny Sto...",neutral,neutral,neutral,neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,K4A7-a5NNRNFT0HnMzpiV68dVpAUX5YmG71Ap5B00vU,"{'name': 'Zacks Investment Research', 'homepag...",Bear Of The Day: Autodesk (ADSK),Brian Bolan,2022-06-28T12:16:00Z,https://www.zacks.com/commentary/1944646/bear-...,[ADSK],https://www.zacks.com/amp/commentary/1944646/b...,https://staticx-tuner.zacks.com/images/default...,Despite beating the Zacks Consensus Estimate i...,NaN,neutral,negative,neutral,negative
996,mJVdMewKY20b0FJzPXDHFQ6-pLZNVJ36z5vZjpdhXE0,"{'name': 'Zacks Investment Research', 'homepag...","Cognyte Software Ltd. (CGNT) Reports Q1 Loss, ...",Zacks Equity Research,2022-06-28T12:15:02Z,https://www.zacks.com/stock/news/1944857/cogny...,"[CGNT, TENB]",https://www.zacks.com/amp/stock/news/1944857/c...,https://staticx-tuner.zacks.com/images/default...,Cognyte Software Ltd. (CGNT) delivered earning...,NaN,negative,neutral,neutral,positive
997,DBfrpkBx33T7oiptfiJHrLZXKaXlytJ3cuXhaWcd6ps,"{'name': 'The Motley Fool', 'homepage_url': 'h...",3 High-Yield REITs That Have Made Me a Ton of ...,newsfeedback@fool.com (Liz Brumer-Smith),2022-06-28T12:15:00Z,https://www.fool.com/investing/2022/06/28/3-hi...,"[PLYM, IIPR, FRT]",NaN,https://g.foolcdn.com/editorial/images/686902/...,The power of dividend growth is severely under...,[investing],neutral,negative,neutral,positive
998,t4RR4xsv1VZU_2xycp7E9ihRyFNAZSxa1hppRVUOUow,"{'name': 'MarketWatch', 'homepage_url': 'https...","Goldman, Wells Fargo lead dividend buffet afte...",MarketWatch,2022-06-28T12:14:00Z,https://www.marketwatch.com/story/goldman-well...,"[